<h1> Подраздел 1.3 <span class="tocSkip"></span></h1>

<h2> Задание 1. Парсинг сайта hh.ru <span class="tocSkip"></span></h2>

In [ ]:
import requests as req 
from bs4 import BeautifulSoup   
import time
import json
import tqdm

In [ ]:
# создадим пустой словарь, куда будем записывать данные
data = {

        "data" : []
    }

In [ ]:

# напишем функцию, которая будет проходить по категориям опыта на сайте hh.ru,
# потом по страницам выдачи от 1 до 120 и парсить необходимые данные

def parsing():
    
    experience = { 'Нет опыта' : 'noExperience',
                  'От 1 года до 3 лет' : 'between1And3',
                  'От 3 до 6 лет' : 'between1And3',
                  'Более 6 лет' : 'moreThan6'
        
    }
    
    

    headers = { "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,/;q=0.8", 
               "User-Agent": "Mozilla/5.0 (Windows NT 10.0; rv:91.0) Gecko/20100101 Firefox/91.0" }

    for exp in range(len(experience)):
        base_url = f"https://spb.hh.ru/search/vacancy?experience={list(experience.values())[exp]}&text=python+разработчик&page="
        
        for page in range(120):
        
            url = base_url+f"{page}"

            resp_text = req.get(url, headers=headers)


            soup = BeautifulSoup(resp_text.text, "lxml") 
            #tags  = soup.find_all(class_="serp-item__title", limit=None)
            tags = soup.find_all(attrs = {"data-qa":"serp-item__title"}, limit=None) 
            #print(tags)
            for tag in tqdm.tqdm(tags):
                time.sleep(5)
                resp_tag = req.get(tag.attrs['href'], headers=headers)
                soup = BeautifulSoup(resp_tag.text, "lxml")

                #tags_object = soup.find(attrs={})

                #tag_experience = soup.find(attrs={"data-qa":"vacancy-salary"}).find(attrs = {"data-qa":"vacancy-experience"}).text

                tag_experience = soup.find(attrs={"data-qa":"vacancy-experience"})
                if tag_experience != None:
                  #print('нашлась')
                  tag_experience = soup.find(attrs={"data-qa":"vacancy-experience"}).text
                tag_price = soup.find(attrs = {"data-qa":"vacancy-serp__vacancy-compensation"})
                if tag_price != None:
                  #print('нашлась')
                  tag_price = soup.find(attrs={"data-qa":"vacancy-serp__vacancy-compensation"}).text
                tag_location = soup.find(attrs = {"data-qa":"vacancy-view-location"})
                if tag_location != None:
                  #print('нашлась')
                  tag_location =  soup.find(attrs = {"data-qa":"vacancy-view-location"}).text

                print(tag.text, tag_experience, tag_price, tag_location)#, tag.attrs['href'])

                data['data'].append({"title" : tag.text, "work_experience":tag_experience, "salary" : tag_price, "region" : tag_location} )

                with open("data.json", "w", encoding="utf-8") as file:
                  json.dump(data,file, ensure_ascii = False)

                #with open(data, 'w', encoding='utf-8') as file:
                    #file.write(json.dumps(data, ensure_ascii=False, indent='\t')) 


In [ ]:
%%time

parsing()